In [ ]:
import numpy as np
import gzip
import matplotlib.pyplot as plt
import datetime

In [ ]:
params = {'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'large',
         'ytick.labelsize':'large'}
plt.rcParams.update(params)

In [ ]:
class UND:
    
    def __init__(self, filename):
        self.filename = filename

    def ReadFile(self):
        
        # Underscored variables are private to the class.
        try:
            # Open the input file for reading.
            # Use the gzip module to read in compressed files.
            if '.gz' in self.filename:
                fhand = gzip.open(self.filename, 'r')
            else:  
                fhand = open(self.filename, 'r')
        except:
            print ('ERROR (readfile.py): Cannot read ASCII file: '+self.filename+', please make sure the file is properly formatted.')


        # Need skiprows to read the right section of the file.
        skiprows  = int(fhand.readline().split()[0])

        # Move file pointer to top of file and grab all header information
        # for parsing.
        fhand.seek(0)
        header = [fhand.readline() for lines in range(skiprows)]

        # Parse the header information.
        # All new-line characters are stripped from strings.
        self.NLHEAD = skiprows
        self.FFI    = int(header[0].split()[1])
        self.ONAME  = header[1].rstrip()
        self.ORG    = header[2].rstrip()
        self.SNAME  = header[3].rstrip()
        self.MNAME  = header[4].rstrip()
        self.IVOL   = int(header[5].split()[0])
        self.VVOL   = int(header[5].split()[1])
        self.DATE   = header[6][:10]
        self.RDATE  = header[6][11:].rstrip()
        self.DX     = float(header[7])
        self.XNAME  = header[8].rstrip()
        self.NV     = int(header[9])
        self.VSCAL  = header[10].split()
        self.VMISS  = header[11].split()
        self.VNAME  = [header[12+i].rstrip() for i in range(self.NV)]
        self.NSCOML = int(header[self.NV+12])
        self.NNCOML = int(header[self.NV+13])
        self.DTYPE  = header[self.NV+14].rstrip()
        self.VFREQ  = header[self.NV+15].rstrip()

        self.VDESC  = header[self.NV+16].split()
        self.VUNITS = header[self.NV+17].split()

        # Read data values from file.
        _data = np.loadtxt(fhand, dtype='float', \
                           skiprows=self.NLHEAD).T
        print(self.VDESC)
        
        

        # Store data in dictionary.
        # Use "object_name.data['Time']" syntax to access data.
        self.data = dict(zip(self.VDESC, _data))
        
        # Close input file.
        fhand.close()
        
    def find_times(self):

        # Find starting and ending times of the file and calculate the total
        # file duration.
        _strt = self.data[self.VDESC[0]][0]
        _endt = self.data[self.VDESC[0]][-1]
        self.start_time = self.sfm2hms(_strt)
        self.end_time   = self.sfm2hms(_endt)     
        
        self.time_hms = []
        for time_s in self.data['Time']:     
            self.time_hms.append(self.sfm2hms(time_s))

        
    def sfm2hms(self, time_s):     
        return str(datetime.timedelta(seconds=round(time_s)))
    


In [ ]:
obj_file = UND(filename='ADPAA/UND_citation/OLYMPEX/15_11_12_18_49_13.olympex')
#obj_file = UND(filename='ADPAA/UND_citation/MC3E/2011_04_25_09_07_51_v2.mc3e')

In [ ]:
obj_file.ReadFile()
#obj_file.data['Time'].min(), obj_file.data['Time'].max()

In [ ]:
obj_file.find_times()
print(obj_file.start_time)
print(obj_file.end_time)

In [ ]:
plt.plot(obj_file.data['Time'], obj_file.data['Air_Temp'])

In [ ]:
time_hms = []
for time in obj_file.data['Time']:
    time_hms.append(str(datetime.timedelta(seconds=round(time))))

In [ ]:
fig, ax1 = plt.subplots(figsize=(15,7))
ax1.plot(time_hms, obj_file.data['REL_HUM'], 'darkblue')
ax1.xaxis.set_ticks(time_hms[0::1000])
#ax1.set_ylim(0, 90200)
ax1.set_xlabel("Time of flight [hh:mm:ss]")
ax1.set_ylabel("Relative Humidity from the EG&G probe [%]", c='darkblue')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.plot(time_hms, obj_file.data['Air_Temp'], 'r')
ax2.xaxis.set_ticks(time_hms[0::1000])
ax2.set_ylabel('Temperature [C]', color='r')
ax2.set_ylim(10, -60)
ax2.axhline(y=0, color='k', linestyle='--')
fig.savefig('ADPAA/plots/olympex_temp_rel_hum.png')
plt.show()

In [ ]:
plt.plot(obj_file.data['Time'], obj_file.data['CDP_LWC'])

In [ ]:
plt.plot(obj_file.data['Time'], obj_file.data['2-DC_EffRad']) #microns

In [ ]:
print(obj_file.VNAME)

In [ ]:
time_hms = []
for time_s in obj_file.data['Time']:     
    time_hms.append(str(datetime.timedelta(seconds=round(time_s))))
time_hms[-10:]

In [ ]:
obj_file.find_times()
obj_file.time_hms

514479_1124-194701_681.png